In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
client = MongoClient('127.0.0.1', 27017)
znss_col = client.Baidu_Robin.znss_comments
sina_col = client.Baidu_Robin.sina_comments
baidu_col = client.Baidu_Robin.baidu_comments

znss = json_normalize(list(znss_col.find()))
sina = json_normalize(list(sina_col.find()))
baidu = json_normalize(list(baidu_col.find()))

In [3]:
znss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4108 entries, 0 to 4107
Columns: 113 entries, _id to user.verified_type_ext
dtypes: bool(9), float64(54), int64(14), object(36)
memory usage: 3.3+ MB


### 1. 数据清洗
- 选取需要的字段
- 按照id去重
- 数据组合

#### 1.1 选取需要的字段

In [5]:
print(list(znss.columns))

['_id', 'bid', 'comments', 'created_at', 'disable_reply', 'floor_number', 'id', 'isLikedByMblogAuthor', 'like_count', 'liked', 'max_id', 'mid', 'more_info_type', 'more_info_users', 'page_info.content1', 'page_info.content2', 'page_info.object_type', 'page_info.page_pic.url', 'page_info.page_title', 'page_info.page_url', 'page_info.type', 'pic.geo.croped', 'pic.geo.height', 'pic.geo.width', 'pic.large.geo.croped', 'pic.large.geo.height', 'pic.large.geo.width', 'pic.large.size', 'pic.large.url', 'pic.pid', 'pic.size', 'pic.url', 'readtimetype', 'rootid', 'rootidstr', 'source', 'text', 'total_number', 'user.avatar_hd', 'user.badge.ali_1688', 'user.badge.anniversary', 'user.badge.asiad_2018', 'user.badge.avengers_2019', 'user.badge.bind_taobao', 'user.badge.cz_wed_2017', 'user.badge.dailv', 'user.badge.dailv_2018', 'user.badge.denglong_2019', 'user.badge.double11_2018', 'user.badge.dzwbqlx_2016', 'user.badge.earth_2019', 'user.badge.follow_whitelist_video', 'user.badge.fools_day_2016', 'us

In [7]:
keep_columns = ['id', 'created_at', 'floor_number', 'text', 'user.id', 'user.screen_name', 
               'user.statuses_count', 'user.gender', 'user.followers_count', 'user.follow_count',
               'total_number', 'like_count']

In [8]:
znss = znss[keep_columns]
sina = sina[keep_columns]
baidu = baidu[keep_columns]

#### 1.2 按照id去重

In [11]:
znss.drop_duplicates(subset='id', inplace=True)
sina.drop_duplicates(subset='id', inplace=True)
baidu.drop_duplicates(subset='id', inplace=True)

In [14]:
znss.shape[0]

3164

In [18]:
print('去重后znss的记录数为'+ str(znss.shape[0]))
print('去重后sina的记录数为'+ str(sina.shape[0]))
print('去重后baidu的记录数为'+ str(baidu.shape[0]))

去重后znss的记录数为3164
去重后sina的记录数为12823
去重后baidu的记录数为11316


In [26]:
znss.sample(5)

,id,created_at,floor_number,text,user.id,user.screen_name,user.statuses_count,user.gender,user.followers_count,user.follow_count,total_number,like_count
1197,4390041789361721,Wed Jul 03 18:49:48 +0800 2019,2469,咋给删完了...,5517150931,SDUFH0,440,m,38,153,0,0
2452,4389991385599134,Wed Jul 03 15:29:31 +0800 2019,1008,咋删了,6590606329,唔喵喵呜,9,m,37,39,1,2
1592,4390023589784599,Wed Jul 03 17:37:28 +0800 2019,2005,做得好兄弟，挺你,3612003823,伊丹英雄,122,m,48,30,0,6
172,4389983005479073,Wed Jul 03 14:56:13 +0800 2019,714,我靠 你的评论被百度公关了,1774956211,Leno宸,41,m,280,173,0,8
2558,4389986747494505,Wed Jul 03 15:11:05 +0800 2019,886,民族英雄！！,5291070263,梁子墨zz,4109,f,209,2593,2,2


In [27]:
znss.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3164 entries, 0 to 3659
Data columns (total 12 columns):
id                      3164 non-null object
created_at              3164 non-null object
floor_number            3164 non-null int64
text                    3164 non-null object
user.id                 3164 non-null int64
user.screen_name        3164 non-null object
user.statuses_count     3164 non-null int64
user.gender             3164 non-null object
user.followers_count    3164 non-null int64
user.follow_count       3164 non-null int64
total_number            3164 non-null int64
like_count              3164 non-null int64
dtypes: int64(7), object(5)
memory usage: 321.3+ KB


#### 1.3 数据组合

In [74]:
znss['source'] = 'znss'
sina['source'] = 'sina'
baidu['source'] = 'baidu'

data = pd.concat([znss, sina, baidu])

In [78]:
# 保存数据
data.to_csv('data.csv', index=False)

In [3]:
data = pd.read_csv('data.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27303 entries, 0 to 27302
Data columns (total 14 columns):
id                      27303 non-null int64
created_at              27303 non-null object
floor_number            27303 non-null int64
text                    27303 non-null object
user.id                 27303 non-null int64
user.screen_name        27303 non-null object
user.statuses_count     27303 non-null int64
user.gender             27303 non-null object
user.followers_count    27303 non-null int64
user.follow_count       27303 non-null int64
total_number            27303 non-null int64
like_count              27303 non-null int64
emoji                   4572 non-null object
source                  27303 non-null object
dtypes: int64(8), object(6)
memory usage: 2.9+ MB


### 2. 问题
- 点赞最高的评论
- 使用最频繁的emoji
- 大家都在讨论的点是什么？
- 百度正在逐渐出局了吗？

In [37]:
znss.loc[znss['like_count'].nlargest(5).index, ['user.id', 'text', 'like_count']]

,user.id,text,like_count
1,1071366672,你这样不太好，你这一浇水，我都不知道李彦宏是干垃圾还是湿垃圾了。,146
2,2120795571,请教一下，传言说，当时您上去将水倒到李彦宏头上的时候，他当时愣了一下，随后百度了一下头上被浇...,139
6,5135499240,兄弟挺住，据可靠消息，李彦宏报警了，选的是百度搜索上排名最高的派出所，目前案件已经进行到向王...,63
5,1898552625,你最好还是清醒一下，也应该庆幸自己没有做出更过激的行为。希望不要被一些三观不正的言论误导，你...,59
4,5462023471,看你的动态 头像 真是个有爱的人 我不认为你从几年前就开始策划这个事情 所以 在这条微博上表...,55


In [6]:
data.loc[data['like_count'].nlargest(10).index, ['user.id', 'text', 'like_count']]

,user.id,text,like_count
15987,1765738452,"官方确认：李彦宏是AI。<span class=""url-icon""><img alt=[d...",41093
3164,1151444033,科普：正确的泼水方式,37489
15988,6593702886,一码归一码，看人被泼水还一片叫好实属卑劣的狂欢。,23234
3165,2182048633,厌恶百度，但这样也不至于。可以抵制，但没必要用暴力。,17925
15958,1935913623,百度这种吸人血馒头的应该泼血才对啊。。。,10255
3166,1280977241,"除了广告审核不严，搜索引擎，导航，地图，以前的MP3,贴吧。我觉得都很好用啊。而且带来很多便...",7801
3167,5748839235,"<span class=""url-icon""><img alt=[允悲] src=""//h5...",7727
15885,1829468941,过分了啊，你再怎么讨厌李彦宏，也不能浪费水啊！,7659
15989,2035302933,泼的是百度吧。 只不过在AI会场,7336
3168,2982859777,我觉得有病吧？！ 一码归一码，这种上去泼水的行为难道不应该拘留？？？？<span class...,6556


#### 2.2 使用最频繁的Emoji

In [7]:
data['emoji'] = data['text'].str.extract("alt=\[(.*?)\]")

In [9]:
top10_emoji = data['emoji'].value_counts()[:10]

In [19]:
from pyecharts import Bar

bar = Bar("评论中最经常使用的Emoji", "总数据量: 27303条", width = 800,height=500)
bar.add("", top10_emoji.index, top10_emoji.values, is_stack=True,bar_category_gap='40%', 
       xaxis_label_textsize=20, yaxis_label_textsize=14, is_label_show=True)
bar

#### 2.3 大家都在讨论的点是什么？

In [28]:
data['text_clean'] = data['text'].str.split('<', expand=True)[0]

In [33]:
import jieba
from collections import Counter
from pyecharts import WordCloud

jieba.add_word('魏则西')
jieba.add_word('李彦宏')
jieba.add_word('莆田')
jieba.add_word('宏颜获水')
jieba.add_word('2块钱')
jieba.add_word('湿垃圾')
jieba.add_word('张全蛋')
jieba.add_word('人血馒头')

swords = [x.strip() for x in open ('/home/kesci/input/baidu_robin6657/stopwords.txt')]

In [34]:
def plot_word_cloud(data, swords):
    text = ''.join(data)
    words = list(jieba.cut(text))
    ex_sw_words = []
    for word in words:
        if len(word)>1 and (word not in swords):
            ex_sw_words.append(word)
    c = Counter()
    c = Counter(ex_sw_words)
    wc_data = pd.DataFrame({'word':list(c.keys()), 'counts':list(c.values())}).sort_values(by='counts', ascending=False).head(100)
    wordcloud = WordCloud(width=1300, height=620)
    wordcloud.add("", wc_data['word'], wc_data['counts'], word_size_range=[20, 100])
    return wordcloud

In [35]:
plot_word_cloud(data['text_clean'], swords)